<a href="https://colab.research.google.com/github/onursercanyilmaz/CarryGO/blob/main/IMDBscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# target : https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250

In [264]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [265]:
#Creating URLs for getting whole links
url_inside=[]
url_soup=[]
original_url = "https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start="
for i in range(0,2288,250):
  url=original_url+str(i)
  url_soup.append(url)

In [3]:
url_soup

['https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=0',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=250',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=500',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=750',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=1000',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=1250',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=1500',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=1750',
 'https://www.imdb.com/search/title/?title_type=tv_series&countries=tr&sort=alpha,asc&count=250&start=2000',
 'https://www.imdb.com/se

In [266]:
#Getting all URLs
for i in url_soup:
  req = requests.get(i)
  soup = BeautifulSoup(req.text, "html.parser")
  x = soup.find(class_="lister-list").find_all("a")
  for j in x:
    if j["href"].__contains__("title") and not (j["href"].__contains__("vote")) and not ((j["href"].__contains__("plotsummary"))) :
      url_inside.append("https://www.imdb.com"+j["href"])
url_inside = list(dict.fromkeys(url_inside))

In [263]:
len(url_inside)
url_inside[0:10]

['https://www.imdb.com/title/tt23109230/',
 'https://www.imdb.com/title/tt10315764/',
 'https://www.imdb.com/title/tt7972832/',
 'https://www.imdb.com/title/tt1809413/',
 'https://www.imdb.com/title/tt13470984/',
 'https://www.imdb.com/title/tt15404372/',
 'https://www.imdb.com/title/tt10313338/',
 'https://www.imdb.com/title/tt11043068/',
 'https://www.imdb.com/title/tt9354484/',
 'https://www.imdb.com/title/tt2544148/']

Get data from list cards

In [267]:

#OUTSIDE
names = []
durations = []
ratings = []
genres = []
certificates = []
stars = []
votes = []
years = []


for i in url_soup:
  req = requests.get(i)
  soup = BeautifulSoup(req.text, "html.parser")
 
  card_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')

  for card in card_containers:
    if card.find("div", class_="lister-item-content") is not None:
#names      
      name = card.h3.a.text
      names.append(name)

#durations
      if card.find("span", class_="runtime") is not None:
        duration = card.find("span", class_="runtime").text.replace(" min", "")
        durations.append(duration)
      else:
         durations.append("None")

#ratings
      if card.find("div", class_="inline-block ratings-imdb-rating") is not None:
        rating = float(card.strong.text)
        ratings.append(rating)
      else:
        ratings.append("None")

#genre
      if card.find("span", class_="genre") is not None:
        genre = card.find("span", class_="genre").text.replace("\n", "").replace(",",".").rstrip().split(',')
        genres.append(genre)
      else:
        genres.append("None")

#certificate
      if card.find("span", class_="certificate") is not None:
        certificate = card.find("span", class_="certificate").text
        certificates.append(certificate)
      else:
        certificates.append("None")

#stars
      if card.select('p')[2].get_text(strip=True) is not None:
        star=(card.select('p')[2].get_text(strip=True).replace("Star:","").replace("Stars:","").replace("Director:","")).split(",")
        stars.append(star)
      else:
        stars.append("None")

#votes      
      if card.find('span', attrs = {'name':'nv'}) is not None:
        vote = int(card.find("span", attrs={"name":"nv"})['data-value'])
        votes.append(vote)
      else:
        votes.append("None")

#years
      if card.find(class_="lister-item-year text-muted unbold").text is not None:
         year= card.find(class_="lister-item-year text-muted unbold").text.replace("(","").replace(")","").split("–")
         years.append(year)
      else:
         years.append("None")

In [268]:
data = pd.DataFrame({'name': names,
                      'duration': durations,
                      'rating': ratings,
                      'genre': genres,
                      'certificate': certificates,
                      'stars': stars,
                      'votes': votes,
                      'years': years}
                      )

In [269]:
data

,name,duration,rating,genre,certificate,stars,votes,years
0,#adaleTT,None,None,[Action],None,"[Can Atak, Yanki Ayhan, Birgen Engin, Zeynep G...",None,[]
1,%100 Futbol,50,5.1,[Sport],None,"[Ridvan Dilmen, Erbatur Ergenekon, Murat Kosov...",11,"[2005, ]"
2,1 Erkek 1 Kadin 2 Cocuk,50,7.6,[Comedy],TV-14,"[Demet Evgar, Emre Karayel]",430,"[2014, 2015]"
3,1 Kadin 1 Erkek,45,7.8,[Comedy],TV-MA,"[Demet Evgar, Emre Karayel, Hare Sürel, Beyaz]",5537,"[2008, 2015]"
4,10 Bin Adim,11,7.8,[Comedy. Drama],None,"[Engin Günaydin, Devin Özgür Çinar, Bugu Selen...",2019,"[2020, ]"
...,...,...,...,...,...,...,...,...
2282,Üzgünüm Leyla,60,5.8,[Comedy. Romance],None,"[Perran Kutman, Selçuk Yöntem, Çetin Tekindor,...",90,"[2000, 2002]"
2283,Üç Arkadas,None,None,[Comedy. Drama. Romance],None,"[Leyla Feray, Hakan Yilmaz, Anil Ilter, Bülent...",None,[2014]
2284,Üç Kiz Kardes,150,4.3,[Drama],None,"[Reha Özcan, Iclal Aydin, Berker Güven, Özgü K...",617,"[2022, ]"
2285,Üç Kurus,120,6.9,[Action. Crime. Drama],None,"[Uraz Kaygilaroglu, Ekin Koç, Aslihan Malbora,...",1558,"[2021, 2022]"


In [ ]:
from datetime import datetime
def stringDateFormat(d):
  if len(d)>4:
    d= d.replace(" (United States)","")
    d_list = d.split(" ")
    d=d_list[0][0:3]+" "+d_list[1]+" "+d_list[2]
    d = datetime.strptime(d, '%b %d, %Y').strftime('%d-%m-%Y')
  return d

In [261]:
"""
#TEST

seasons = []

i="https://www.imdb.com/title/tt8090594/?ref_=adv_li_tt"
req = requests.get(i)
soup = BeautifulSoup(req.text, "html.parser")


#seasons
if soup.find("div", class_="ipc-simple-select ipc-simple-select--base ipc-simple-select--theme-base ipc-simple-select--on-accent2") is not None:
  if  soup.find("select", attrs={"id":"browse-episodes-year"}) is not None:
    season = soup.find("select", attrs={"id":"browse-episodes-year"})["aria-label"].replace(" years","")
    seasons.append(season)
  else:
      seasons.append("None")
elif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find( class_="sc-93b8eec8-4 dDCqHi"))==3:
  season = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find_all(class_="ipc-button__text")[1].text.replace(" Season","")
  seasons.append(season)
elif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find(class_="sc-93b8eec8-4 dDCqHi"))==2:
  season = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find(class_="ipc-button__text").text.replace(" Season","")
  seasons.append(season)
else:
  seasons.append("None")

"""

'\n#TEST\n\nseasons = []\n\ni="https://www.imdb.com/title/tt8090594/?ref_=adv_li_tt"\nreq = requests.get(i)\nsoup = BeautifulSoup(req.text, "html.parser")\n\n\n#seasons\nif soup.find("div", class_="ipc-simple-select ipc-simple-select--base ipc-simple-select--theme-base ipc-simple-select--on-accent2") is not None:\n  if  soup.find("select", attrs={"id":"browse-episodes-year"}) is not None:\n    season = soup.find("select", attrs={"id":"browse-episodes-year"})["aria-label"].replace(" years","")\n    seasons.append(season)\n  else:\n      seasons.append("None")\nelif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find( class_="sc-93b8eec8-4 dDCqHi"))==3:\n  season = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find_all(class_="ipc-button__text")[1].text.replace(" Season","")\n  seasons.append(season)\nelif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find(class_="sc-93b8eec8-4 dDCqHi"))==2:\n  season = soup.find("div", class_="sc-93

In [260]:
#INSIDE
summaries = []
episodes = []
awards = []
seasons = []
release_dates = []
soundtracks = []
soundtrack_comments = []
filming_locations = []
production_companies = []
trivias = []
connections = []
posters = []
total_years = []

for i in url_inside:
  req = requests.get(i)
  soup = BeautifulSoup(req.text, "html.parser")

#summaries
  if soup.find(class_="sc-16ede01-2 gXUyNh") is not None:
    summary = soup.find(class_="sc-16ede01-2 gXUyNh").text
    summaries.append(summary)
  else:
    summaries.append("None")

#episodes
  if soup.find("span", class_="ipc-title__subtext") is not None:
    episode = soup.find("span", class_="ipc-title__subtext").text
    episodes.append(episode)
  else:
    episodes.append("None")

#awards
  if soup.find("div", attrs={"data-testid":"awards"}) is not None:
    award = soup.find("div", attrs={"data-testid":"awards"}).find("label").text.replace("wins ","").replace(" nominations","").split(" & ")
    awards.append(awards)
  else:
    awards.append("None")

#seasons
  if soup.find("div", class_="ipc-simple-select ipc-simple-select--base ipc-simple-select--theme-base ipc-simple-select--on-accent2") is not None:
    if  soup.find("select", attrs={"id":"browse-episodes-season"}) is not None:
      season = soup.find("select", attrs={"id":"browse-episodes-season"})["aria-label"].replace(" seasons","")
      seasons.append(season)
    else:
       seasons.append("None")
  elif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find( class_="sc-93b8eec8-4 dDCqHi"))==3:
    season = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find_all(class_="ipc-button__text")[1].text.replace(" Season","")
    seasons.append(season)
  elif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find(class_="sc-93b8eec8-4 dDCqHi"))==2:
    season = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find(class_="ipc-button__text").text.replace(" Season","")
    seasons.append(season)
  else:
    seasons.append("None")

#release_date
  if soup.find("li", attrs={"data-testid":"title-details-releasedate"}) is not None:
    release_date = soup.find("li", attrs={"data-testid":"title-details-releasedate"}).find("li").find("a").text.replace(" (Turkey)","")
    #release_date = stringDateFormat(release_date)
    print(release_date)
    release_dates.append(release_date)
  else:
    release_dates.append("None")

#soundtracks  
  if soup.find("span", attrs={"class":"soundtrack-trackname"}) is not None:
    soundtrack = soup.find("span", attrs={"class":"soundtrack-trackname"}).text
    soundtracks.append(soundtrack)
  else:
    soundtracks.append("None")

#soundtrack_comments
  if soup.find("span", attrs={"class":"soundtrack-comment-1"}) is not None:
    soundtrack_comment = soup.find("span", attrs={"class":"soundtrack-comment-1"}).text
    soundtrack_comments.append(soundtrack_comment)
  else:
    soundtrack_comments.append("None")

#filming_locations
  if soup.find("li", attrs={"data-testid":"title-details-filminglocations"}) is not None:
    filming_location = soup.find("li", attrs={"data-testid":"title-details-filminglocations"}).find(class_="ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link").text
    filming_locations.append(filming_location)
  else:
    filming_locations.append("None")

#production_companies
  if soup.find("li", attrs={"data-testid":"title-details-companies"}) is not None:
    production_company = soup.find("li", attrs={"data-testid":"title-details-companies"}).find_all("li")
    for x in production_company:
      pc = x.find("a").text
      production_companies.append(pc)
  else:
    production_companies.append("None")
  
#trivias
  if soup.find(class_="ipc-overflowText ipc-overflowText--listCard ipc-overflowText--height-short ipc-overflowText--short ipc-overflowText--click ipc-overflowText--base") is not None:
    trivia = soup.find(class_="ipc-overflowText ipc-overflowText--listCard ipc-overflowText--height-short ipc-overflowText--short ipc-overflowText--click ipc-overflowText--base").text
    trivias.append(trivia)
    print(trivia)
  else:
    trivias.append("None")

#connections
  try:
    if soup.find("section", attrs={"data-testid":"DidYouKnow"}) is not None and soup.find_all("div", attrs={"class":"ipc-list-card--border-line ipc-list-card--tp-none ipc-list-card--bp-none ipc-list-card sc-74d1a1fd-1 euRjsE ipc-list-card--base"}) is not None:
      if len(soup.find("section", attrs={"data-testid":"DidYouKnow"})) == 4 or len(soup.find("section", attrs={"data-testid":"DidYouKnow"})) == 3:
        conn = soup.find_all("div", attrs={"class":"ipc-list-card--border-line ipc-list-card--tp-none ipc-list-card--bp-none ipc-list-card sc-74d1a1fd-1 euRjsE ipc-list-card--base"})[1].ul.li.a.text == "Connections"
        if len(soup.find("section", attrs={"data-testid":"DidYouKnow"})) == 4 and conn :
          section = soup.find("section", attrs={"data-testid":"DidYouKnow"})
          connection = section.find_all(class_="ipc-html-content ipc-html-content--base sc-74d1a1fd-0 gbUKeh")
          connection = connection[1].text
          connections.append(connection)
        elif len(soup.find("section", attrs={"data-testid":"DidYouKnow"})) == 3  and conn :
          section = soup.find("section", attrs={"data-testid":"DidYouKnow"})
          connection = section.find_all(class_="ipc-html-content ipc-html-content--base sc-74d1a1fd-0 gbUKeh")
          connection = connection[1].text
          connections.append(connection)
        else:
          connections.append("None")
      else:
          connections.append("None")

    else:
      connections.append("None")
  except:
    connections.append("None")

#images
  if soup.find("img", class_="ipc-image") is not None:
    poster = soup.find("img", class_="ipc-image").get("src")
    posters.append(poster)
  else:
    posters.append("None")

#total_years
if soup.find("div", class_="ipc-simple-select ipc-simple-select--base ipc-simple-select--theme-base ipc-simple-select--on-accent2") is not None:
  if  soup.find("select", attrs={"id":"browse-episodes-year"}) is not None:
    total_year = soup.find("select", attrs={"id":"browse-episodes-year"})["aria-label"].replace(" years","")
    total_years.append(total_year)
  else:
      total_years.append("None")
elif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find( class_="sc-93b8eec8-4 dDCqHi"))==3:
  total_year = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find_all(class_="ipc-button__text")[1].text.replace(" Season","")
  total_years.append(total_year)
elif soup.find("div", class_="sc-93b8eec8-4 dDCqHi") is not None and len(soup.find(class_="sc-93b8eec8-4 dDCqHi"))==2:
  total_year = soup.find("div", class_="sc-93b8eec8-4 dDCqHi").find(class_="ipc-button__text").text.replace(" Season","")
  total_years.append(total_year)
else:
  total_years.append("None")


2005
December 3, 2014
2008
The show is aired on Tuesday, Thursday and Saturday evenings at 22:00 at TurkMax.
December 30, 2020
September 10, 2021
April 3, 1994
January 1, 2013
The story is based on the French thriller "Ohne Schuld (2008)", directed by "Fred Cavayé".
August 20, 2010
September 9, 2011
August 17, 2012
At the end of the season, Galatasaray became the champion.
August 16, 2013
At the end of the season, Fenerbahce became the champion.
August 17, 2013
August 29, 2014
At the end of the season, Bursaspor's footballer Fernandão became top scorer. (22 Goals)
August 29, 2014
August 14, 2015
At the end of the season, Besiktas's footballer Mario Gómez became top scorer. (26 Goals)
August 14, 2015
August 19, 2016
The season was named after Turgay Seren, a former Turkish national team player and Galatasaray legend.
August 19, 2016
August 11, 2017
The season was named after Ilhan Cavcav, a former Turkish businessman and Genclerbirligi legend president.
August 11, 2017
August 10, 2018
T

KeyboardInterrupt: ignored

# **Merge two dataset by depending on tv series name**

'TV-14'

In [ ]:
# get all data from URLs
all_data = []
for url in url_inside:
   

    req = requests.get(url)
    soup = BeautifulSoup(req.text, "html.parser")

    h2 = soup.h2
    sub = h2.find_next("p")

    for tr in soup.select("tr:has(td)"):
        all_data.append(
            [
                h2.get_text(strip=True),
                sub.get_text(strip=True),
                *[td.get_text(strip=True) for td in tr.select("td")],
            ]
        )

# save data to CSV
df = pd.DataFrame(
    all_data,
    columns=[
        "title",
        "sub_title",
        "Rank",
        "Horse / Owner",
        "Points",
        "Total Comps",
    ],
)
print(df)
df.to_csv("data.csv", index=None)

KeyboardInterrupt: ignored